In [21]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import mxnet as mx

from joblib import Parallel, delayed
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
patches_rootdir = '/home/yuncong/CSHL_data_patches/'

model_dir = '/home/yuncong/mxnet_models/'
# model_name='inception-stage1'
# model_name = 'experiment0227'
# model_iteration = 6

model_name = 'experiment0317'
model_iteration = 6

model = mx.model.FeedForward.load(os.path.join(model_dir, model_name), model_iteration, ctx=mx.gpu())

model.arg_params['fullc_bias'].asnumpy()

array([ -2.80252025e-02,   4.67557786e-03,   1.94432028e-02,
         2.13614497e-02,   3.16425064e-03,   1.62128229e-02,
        -6.66866952e-04,  -2.80861207e-03,  -3.22492816e-03,
        -5.51616680e-03,  -9.06719640e-03,  -3.54129998e-06,
         5.71480673e-03,  -5.72868949e-03,  -5.52872987e-03,
        -1.00020496e-02], dtype=float32)

In [23]:
# fc_output = model.symbol.get_internals()['fc_output']
flatten_output = model.symbol.get_internals()['flatten_output']
# fc_output = model.symbol.get_internals()['fullc_output']
sm_output = model.symbol.get_internals()['softmax_output']
grouped_output = mx.symbol.Group([flatten_output, sm_output])

model = mx.model.FeedForward(ctx=mx.gpu(), symbol=grouped_output, num_epoch=model_iteration,
                            arg_params=model.arg_params, aux_params=model.aux_params,
                            allow_extra_params=True)

In [24]:
# labels =  ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL']

# labels = ['BackG', '5N', '7n', '7N', '12N', 'Gr', 'LVe', 'Pn', 'SuVe', 'VLL', 
#                      '6N', 'Amb', 'R', 'Tz', 'Sol', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

labels = ['BackG', '5N', '7n', '7N', '12N', 'Pn', 'VLL', 
          '6N', 'Amb', 'R', 'Tz', 'RtTg', 'LRt', 'LC', 'AP', 'sp5']

label_dict = dict([(l,i) for i, l in enumerate(labels)])

# label_dict = dict([(l,i) for i, l in enumerate(labels)] + \
#                   zip(other_labels, range(len(labels), len(labels)+len(other_labels))))

In [25]:
from scipy.interpolate import RectBivariateSpline
from skimage.transform import resize

In [26]:
# scoremaps_rootdir = '/home/yuncong/CSHL_scoremaps_lossless'
# if not os.path.exists(scoremaps_rootdir):
#     os.makedirs(scoremaps_rootdir)
    
predictions_rootdir = '/home/yuncong/CSHL_patch_predictions'
if not os.path.exists(predictions_rootdir):
    os.makedirs(predictions_rootdir)

# iterators_rootdir = '/home/yuncong/CSHL_mxnet_iterators'
# if not os.path.exists(iterators_rootdir):
#     os.makedirs(iterators_rootdir)

In [27]:
mean_img = mx.nd.load(os.path.join(model_dir, 'mean_224.nd'))['mean_img'].asnumpy()

In [28]:
progress_bar = FloatProgress(min=0, max=np.inf)

In [ ]:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
# for stack in ['MD591']:
# for stack in ['MD591', 'MD595', 'MD598', 'MD602', 'MD594']:
for stack in ['MD589']:
# for stack in ['MD589']:

    dm = DataManager(stack=stack)
    
    first_bs_sec, last_bs_sec = section_range_lookup[stack]
    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]
    
    progress_bar.min = first_detect_sec
    progress_bar.max = last_detect_sec
    display(progress_bar)
    
    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    if stack in ['MD589', 'MD594']:
        stack_has_annotation = True
    else:
        stack_has_annotation = False

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')
    
    for sec in range(first_detect_sec, last_detect_sec+1):
#     for sec in range(first_detect_sec, first_detect_sec+1):
        
        if sec not in indices_allROIs_allSections.columns:
            continue

        print sec

        progress_bar.value = sec

        indices_roi = indices_allROIs_allSections[sec]['roi1']

        predictions_dir = os.path.join(predictions_rootdir, stack, '%04d'%sec)
        if not os.path.exists(predictions_dir):
            os.makedirs(predictions_dir)

        ## define grid, generate patches

        t = time.time()
        
        dm.set_slice(sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]
        sample_locations_roi = sample_locations[indices_roi]

        n = len(indices_roi)
        print n, 'samples'
        
        patches = np.asarray([dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
                              for x, y in sample_locations_roi])

        t = time.time()

        patches = np.rollaxis(patches, 3, 1)
        patches = patches - mean_img

        dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

        true_labels = -1 * np.ones((99999,), np.int)
        if stack_has_annotation:
            if sec in indices_allLandmarks_allSections:
                for l in indices_allLandmarks_allSections[sec].dropna().keys():
                    if l == 'bg': continue
                    if l.endswith('surround'):
                        true_labels[indices_allLandmarks_allSections[sec][l]] = 0
                    else:
                        if l in labels:
                            true_labels[indices_allLandmarks_allSections[sec][l]] = label_dict[l]
                        else:
                            true_labels[indices_allLandmarks_allSections[sec][l]] = 0
    
        patch_labels = true_labels[indices_roi]
                
        np.save(predictions_dir+'/%(dataset)s_patchLabels.npy' % {'dataset': dataset}, patch_labels)
        
#         sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))
                
        # feed the network
        
#         continue
#         t = time.time()
        
        batch_size = 512 # increasing to 892 does not save any time
        
        test_iter = mx.io.NDArrayIter(
            patches, 
            patch_labels,
            batch_size = batch_size,
            shuffle=False
        )

#         sys.stderr.write('load iterator: %.2f seconds\n' % (time.time() - t))

        t = time.time()
        
        output = model.predict(test_iter)
        
        features = output[0]
        probs = output[1] # 0 for flatteed features, 1 for softmax output
        
        bp.pack_ndarray_file(features, predictions_dir + '/%(dataset)s_features.bp'% {'dataset': dataset})
#         np.save(predictions_dir + '/%(dataset)s_features.npy'% {'dataset': dataset}, features)
        np.save(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset}, probs)
        
        # probs = np.load(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset})
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))


150
19064 samples
151
19328

In [38]:
np.unique(np.argmax(probs, axis=1))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15])

In [ ]:
# Use RecordIO - dump to images, pack into records, then use ImageRecordIte - 80s for gen patch, 80s for packing

In [ ]:
# for stack in ['MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602']:
for stack in ['MD585']:

    first_bs_sec, last_bs_sec = section_range_lookup[stack]

    table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allROIs_allSections.h5'%{'stack':stack})
    indices_allROIs_allSections = pd.read_hdf(table_filepath, 'indices_allROIs_allSections')
    grid_parameters = pd.read_hdf(table_filepath, 'grid_parameters')

    first_detect_sec, last_detect_sec = detect_bbox_range_lookup[stack]

    stack_has_annotation = False

    if stack_has_annotation:
        table_filepath = os.path.join(patches_rootdir, '%(stack)s_indices_allLandmarks_allSections.h5'%{'stack':stack})
        indices_allLandmarks_allSections = pd.read_hdf(table_filepath, 'indices_allLandmarks_allSections')

#     for sec in range(first_detect_sec, last_detect_sec+1):
    for sec in [first_detect_sec]:

        if sec not in indices_allROIs_allSections.columns:
            continue

        print sec

        indices_roi = indices_allROIs_allSections[sec]['roi1']

        predictions_dir = os.path.join(predictions_rootdir, stack, '%04d'%sec)
        if not os.path.exists(predictions_dir):
            os.makedirs(predictions_dir)

        iterators_dir = os.path.join(iterators_rootdir, stack, '%04d'%sec)
        if not os.path.exists(iterators_dir):
            os.makedirs(iterators_dir)

        scoremaps_dir = os.path.join(scoremaps_rootdir, stack, '%04d'%sec)
        if not os.path.exists(scoremaps_dir):
            os.makedirs(scoremaps_dir)

        patches_dir = os.path.join(patches_rootdir, '%s_byROI/%04d/roi1'%(stack, sec))
        if not os.path.exists(patches_dir):
            os.makedirs(patches_dir)

        ## define grid, generate patches

        t = time.time()

        dm = DataManager(stack=stack, section=sec)
        dm._load_image(['rgb-jpg'])

        patch_size, stride, w, h = grid_parameters.tolist()
        half_size = patch_size/2

        ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                         indexing='xy')

        sample_locations = np.c_[xs.flat, ys.flat]
        sample_locations_roi = sample_locations[indices_roi]

        print len(indices_roi), 'samples'

#         for i, (x,y) in zip(indices_roi, sample_locations_roi):
#             patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
#             cv2.imwrite(os.path.join(patches_dir, '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' %\
#                                      {'stack':stack, 'sec':sec, 'gridp_ind':i}), 
#                         patch[..., ::-1])

        sys.stderr.write('generating patches: %.2f seconds\n' % (time.time() - t))

        #     ## predict for the patches

        dataset = '%(stack)s_%(sec)04d_roi1' % {'stack': stack, 'sec': sec}

        img_filenames = sorted(os.listdir(patches_dir))

        true_labels = -1 * np.ones((99999,), np.int)
        if stack_has_annotation:
            if sec in indices_allLandmarks_allSections:
                for l in indices_allLandmarks_allSections[sec].dropna().keys():
                    if l == 'bg': continue
                    if l.endswith('surround'):
                        true_labels[indices_allLandmarks_allSections[sec][l]] = 0
                    else:
                        true_labels[indices_allLandmarks_allSections[sec][l]] = label_dict[l]

        with open(iterators_dir + '/%(dataset)s_test.lst' % {'dataset': dataset}, 'w') as lst_file:
            for index, img_filename in enumerate(img_filenames):
                grid_index = int(img_filename[:-4].split('_')[-1])
                lst_file.write('%s\t%d\t%s\n'%(index, true_labels[grid_index], img_filename))

        t = time.time()

        cmd_gen_iterator = os.environ['MXNET_DIR']+'/bin/im2rec %(iterators_dir)s/%(dataset)s_test.lst \
        %(data_dir)s/ %(iterators_dir)s/%(dataset)s_test.rec' % \
                  {'dataset': dataset, 'data_dir': patches_dir, 'iterators_dir': iterators_dir}
        # note: the / after data_dir is necessary

#         os.system(cmd_gen_iterator)

        sys.stderr.write('generating iterators: %.2f seconds\n' % (time.time() - t))

        # feed the network
        batch_size = 256

        t = time.time()
        
        test_iter = mx.io.ImageRecordIter(
            path_imgrec = iterators_dir + '/%(dataset)s_test.rec' % {'dataset': dataset},
            batch_size = batch_size,
            data_shape = (3, 224, 224),
            mean_img = os.path.join(model_dir, 'mean_224.nd'),
        )

        sys.stderr.write('load iterators: %.2f seconds\n' % (time.time() - t))

        
        t = time.time()
        probs = model.predict(test_iter)
        np.save(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset}, probs)
        # probs = np.load(predictions_dir + '/%(dataset)s_predictions.npy'% {'dataset': dataset})
        sys.stderr.write('predict: %.2f seconds\n' % (time.time() - t))

        break
        
        ## interpolate

        interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
        interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
        interpolation_w = interpolation_xmax - interpolation_xmin + 1
        interpolation_h = interpolation_ymax - interpolation_ymin + 1

        ##### sample_locations_roi + scores to dense_score_map #####

        shrink_factor = 4

        sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
        sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])

        n_sample_x = sample_locations_unique_xs.size
        n_sample_y = sample_locations_unique_ys.size

        index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
        index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
        sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])

        sample_locations_interpolatedArea_ys_matrix, \
        sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                        interpolation_ymax/shrink_factor), 
                                                                  range(interpolation_xmin/shrink_factor, 
                                                                        interpolation_xmax/shrink_factor), 
                                                                  indexing='ij')


        def generate_score_map(l):

            label = labels[(l+1)%len(labels)]
            if label == 'BackG':
                return

            score_matrix = np.zeros((n_sample_x, n_sample_y))
            score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs[:,l]

            spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                         sample_locations_unique_ys/shrink_factor, 
                                         score_matrix, 
                                         bbox=[interpolation_xmin/shrink_factor, 
                                               interpolation_xmax/shrink_factor, 
                                               interpolation_ymin/shrink_factor, 
                                               interpolation_ymax/shrink_factor])

            dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, 
                                        sample_locations_interpolatedArea_ys_matrix)

            dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w))


            dense_score_map[dense_score_map < 1e-2] = 0
            dense_score_map_lossless = np.pad(dense_score_map, ((interpolation_ymin, h-interpolation_ymax-1), 
                                                  (interpolation_xmin, w-interpolation_xmax-1)),
                                              mode='constant', constant_values=0)
            
            if np.count_nonzero(score_map[::8, ::8] > .1) < 1e3:
                sys.stderr.write('No %s is detected on section %d\n' % (label, sec))
            
#             bp.pack_ndarray_file(dense_score_map_lossless, 
#                                    os.path.join(scoremaps_dir, '%(dataset)s_denseScoreMapLossless_%(label)s.bp' % \
#                                                 {'dataset': dataset, 'label': label}))


        t = time.time()

        n_labels = probs.shape[1]

        _ = Parallel(n_jobs=12)(delayed(generate_score_map)(l) for l in range(n_labels))

        sys.stderr.write('interpolate: %.2f seconds\n' % (time.time() - t))